In [14]:
import requests
import json
from datetime import datetime, timedelta
import smtplib
import os
from twilio.rest import Client
import time

In [55]:
#generate an empty list to store appointment times in
appt_list = []
print("list:  ", appt_list)

list:   []


In [37]:
#check for appointments from fairbank and add to list if new
d_API = requests.get('https://ttp.cbp.dhs.gov/schedulerapi/slots?orderBy=soonest&limit=1&locationId=6480&minimum=1')
a = json.loads(d_API.text)

if len(a) == 0:
    pass
    print("skipping, list is empty")
else:
    appt_time = a[0]['startTimestamp']
    t = datetime.strptime(appt_time, '%Y-%m-%dT%H:%M')


skipping, list is empty


In [56]:
def periodic_work(interval):
    while True:

        #check for appointments from fairbank and add to list if new
        d_API = requests.get('https://ttp.cbp.dhs.gov/schedulerapi/slots?orderBy=soonest&limit=1&locationId=6480&minimum=1')
        a = json.loads(d_API.text)
        if len(a) == 0:
            pass
            print("No new appointments")
        else:
            appt_time = a[0]['startTimestamp']
            t = datetime.strptime(appt_time, '%Y-%m-%dT%H:%M')
            if t not in appt_list:
                appt_list.append(t)
                print("New appointment added on", t.strftime("%A %B %d, %Y"), "sending a text alert now...")
                account_sid = 'AC59fe854bae06cfdbe5f0aae5375a7be7'
                auth_token = '73af7c98667b85967c90e0b9b24eeb2a'
                client = Client(account_sid, auth_token)
                message = client.messages \
                            .create(
                                 body="New Bowling Green appointment added on {} at {}".format(t.strftime("%A %B %d, %Y"),t.strftime("%H:%M %p")),
                                 from_='+19794067684',
                                 to='+16073425655'
                             )
            else:
                print("No new appointments")

        
        #interval should be an integer, the number of seconds to wait
        time.sleep(interval)

In [57]:
periodic_work(15)

New appointment added on Wednesday July 13, 2022 sending a text alert now...
No new appointments


KeyboardInterrupt: 